# Bengaluru House Price Prediction

### Importing the Necessary Libraries

In [244]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn.svm import SVR

print("All imports are working perfectly.")



All imports are working perfectly.


### Reading the Dataset

In [245]:
df = pd.read_csv(r"..\dataset\Bengaluru_House_Data.csv")

In [246]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [247]:
df.shape

(13320, 9)

In [248]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


In [249]:
df.describe()

,bath,balcony,price
count,13247.000000,12711.000000,13320.000000
mean,2.692610,1.584376,112.565627
std,1.341458,0.817263,148.971674
min,1.000000,0.000000,8.000000
25%,2.000000,1.000000,50.000000
50%,2.000000,2.000000,72.000000
75%,3.000000,2.000000,120.000000
max,40.000000,3.000000,3600.000000


### Checking for Null Values

In [250]:
df.isna().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

### Dropping the Unnecessary Columns

In [251]:
df.drop(columns=['area_type','availability','society','balcony'],inplace=True)

In [252]:
df.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

### Checking for Duplicate Values


In [253]:
df.duplicated().sum()

np.int64(882)

### Dropping the Duplicate Rows

In [254]:
df.drop_duplicates(inplace=True)

In [255]:
df.duplicated().sum()

np.int64(0)

In [256]:
df['location'].value_counts()

location
Whitefield          507
Sarjapur  Road      364
Electronic City     273
Thanisandra         224
Kanakpura Road      223
                   ... 
Udayagiri             1
pavitra paradise      1
kadubisnahalli        1
Rahmath Nagar         1
Chikkanahalli         1
Name: count, Length: 1305, dtype: int64

In [257]:
df.location.isnull().sum()

np.int64(1)

## Filling the missing value in location column by Whitefield(as it's the most common loaction)                     

In [258]:
df['location'] = df['location'].fillna("Whitefield")

In [259]:
df['size'].value_counts()

size
2 BHK         4768
3 BHK         3988
4 Bedroom      819
4 BHK          562
3 Bedroom      527
1 BHK          507
2 Bedroom      303
5 Bedroom      288
6 Bedroom      191
1 Bedroom      100
8 Bedroom       84
7 Bedroom       82
5 BHK           59
9 Bedroom       46
6 BHK           30
7 BHK           17
1 RK            12
10 Bedroom      12
9 BHK            8
8 BHK            5
11 BHK           2
10 BHK           2
11 Bedroom       2
27 BHK           1
19 BHK           1
43 Bedroom       1
16 BHK           1
14 BHK           1
12 Bedroom       1
13 BHK           1
18 Bedroom       1
Name: count, dtype: int64

### Filling the missing values in the size column by 2BHK 

In [260]:
df['size'] = df['size'].fillna("2 BHK")

In [261]:
df['size'].isnull().sum()

np.int64(0)

### Filling the missing values in the bathroom column by it's median

In [262]:
df['bath'] = df['bath'].fillna(df['bath'].median())

In [263]:
df['bath'].isnull().sum()

np.int64(0)

In [264]:
df.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

#### No more missing values present


### Since the size column contains discrepancy so we will filter the column and store it in another column

In [265]:
df['BHK'] = df['size'].str.split().str.get(0).astype('int')

In [266]:
df

,location,size,total_sqft,bath,price,BHK
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2
...,...,...,...,...,...,...
13314,Green Glen Layout,3 BHK,1715,3.0,112.00,3
13315,Whitefield,5 Bedroom,3453,4.0,231.00,5
13316,Richards Town,4 BHK,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,2 BHK,1141,2.0,60.00,2


### Dropping the size column as it is no longer needed

In [267]:
df.drop(columns=['size'],inplace=True)

In [268]:
df['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

### Since this column contains range, which is not desired so we will take the mean value of the range

In [269]:
def convertrange(x):
    temp = x.split('-')
    if len(temp) == 2:
        return ((float(temp[0])+float(temp[1]))/2)
    try:
        return float(x)
    except:
        return None

In [270]:
df['total_sqft'] = df['total_sqft'].apply(convertrange)

In [271]:
df

,location,total_sqft,bath,price,BHK
0,Electronic City Phase II,1056.0,2.0,39.07,2
1,Chikka Tirupathi,2600.0,5.0,120.00,4
2,Uttarahalli,1440.0,2.0,62.00,3
3,Lingadheeranahalli,1521.0,3.0,95.00,3
4,Kothanur,1200.0,2.0,51.00,2
...,...,...,...,...,...
13314,Green Glen Layout,1715.0,3.0,112.00,3
13315,Whitefield,3453.0,4.0,231.00,5
13316,Richards Town,3600.0,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141.0,2.0,60.00,2


In [272]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12438 entries, 0 to 13318
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    12438 non-null  object 
 1   total_sqft  12392 non-null  float64
 2   bath        12438 non-null  float64
 3   price       12438 non-null  float64
 4   BHK         12438 non-null  int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 583.0+ KB


### Creating a new column for price per square feet

In [273]:
df['price_per_sqft'] = (df['price']*100000)/ df ['total_sqft']

In [274]:
df['price_per_sqft']

0         3699.810606
1         4615.384615
2         4305.555556
3         6245.890861
4         4250.000000
             ...     
13314     6530.612245
13315     6689.834926
13316    11111.111111
13317     5258.545136
13318    10407.336319
Name: price_per_sqft, Length: 12438, dtype: float64

In [275]:
df.describe()

,total_sqft,bath,price,BHK,price_per_sqft
count,12392.000000,12438.000000,12438.000000,12438.000000,1.239200e+04
mean,1576.376848,2.715549,115.408159,2.826821,8.099928e+03
std,1273.831772,1.367021,153.069125,1.323065,1.101479e+05
min,1.000000,1.000000,8.000000,1.000000,2.678298e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.306941e+03
50%,1288.500000,2.000000,74.000000,3.000000,5.500000e+03
75%,1700.000000,3.000000,125.000000,3.000000,7.446809e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


In [276]:
df['location'].value_counts()

location
Whitefield          508
Sarjapur  Road      364
Electronic City     273
Thanisandra         224
Kanakpura Road      223
                   ... 
Udayagiri             1
pavitra paradise      1
kadubisnahalli        1
Rahmath Nagar         1
Chikkanahalli         1
Name: count, Length: 1305, dtype: int64

In [277]:
df['location'] = df['location'].apply(lambda x: x.strip())


In [278]:
location_count = df['location'].value_counts()

In [279]:
location_count_less10 = location_count[location_count<=10]
location_count_less10

location
Vasanthapura             10
Gunjur Palya             10
Dodsworth Layout         10
1st Block Koramangala    10
Kalkere                  10
                         ..
Hsr layout sector3        1
Thyagraj Nagar            1
Banagirinagar             1
Medaralli                 1
Chikkajala                1
Name: count, Length: 1063, dtype: int64

In [280]:
df['location'] = df['location'].apply(lambda x : 'other' if x in location_count_less10 else x)

In [281]:
df['location'].value_counts()

location
other                   2927
Whitefield               509
Sarjapur  Road           364
Electronic City          275
Thanisandra              226
                        ... 
Tindlu                    11
Banashankari Stage V      11
Doddakallasandra          11
Thyagaraja Nagar          11
HAL 2nd Stage             11
Name: count, Length: 232, dtype: int64

In [282]:
df = df[((df['total_sqft']/df['BHK'])>=300)]

### Removing Outliers

In [283]:
def remove_outlier_sqft(df):
    df_output = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        gen_df = subdf[(subdf.price_per_sqft>(m-st))& (subdf.price_per_sqft<=(m+st))]
        df_output = pd.concat([df_output,gen_df],ignore_index=True)
    return df_output

df = remove_outlier_sqft(df)

In [284]:
def bhk_outlier_remover(df):
    exclude_indices = np.array([])

    # Group by location
    for location, location_df in df.groupby('location'):
        bhk_stats = {}

        # Group by BHK within each location
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                "mean": np.mean(bhk_df['price_per_sqft']),
                "std": np.std(bhk_df['price_per_sqft']),
                "count": bhk_df.shape[0]
            }

        # Identify outliers
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk - 1)
            if stats and stats['count'] > 5:
                exclude_indices = np.append(
                    exclude_indices,
                    bhk_df[bhk_df['price_per_sqft'] < (stats['mean'])].index.values
                )

    # Drop outliers
    return df.drop(exclude_indices, axis='index')

In [285]:
df.shape

(9606, 6)

In [286]:
df

,location,total_sqft,bath,price,BHK,price_per_sqft
0,1st Block Jayanagar,2850.0,4.0,428.00,4,15017.543860
1,1st Block Jayanagar,1630.0,3.0,194.00,3,11901.840491
2,1st Block Jayanagar,1875.0,2.0,235.00,3,12533.333333
3,1st Block Jayanagar,1200.0,2.0,130.00,3,10833.333333
4,1st Block Jayanagar,1235.0,2.0,148.00,2,11983.805668
...,...,...,...,...,...,...
9601,other,1800.0,1.0,200.00,1,11111.111111
9602,other,1095.0,2.0,57.00,2,5205.479452
9603,other,1440.0,2.0,63.93,3,4439.583333
9604,other,1075.0,2.0,48.00,2,4465.116279


### Removing the Unnecessary Columns

In [287]:
df.drop(columns=['price_per_sqft'],inplace=True)

In [288]:
df.head()

,location,total_sqft,bath,price,BHK
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2


## Saving the cleaned data in .csv format

In [289]:
df.to_csv("..\dataset\cleaned_house_data.csv")

## X, Y split

In [290]:
x = df.drop(columns=['price'])
y = df['price']

## Train test split

In [291]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [292]:
x_train.shape,x_test.shape

((7684, 4), (1922, 4))

### Encoding the location column

In [293]:
colmn_trans = make_column_transformer(
    (OneHotEncoder(drop='if_binary'), ['location']),  # OneHotEncode 'location' column
    remainder='passthrough'  # Keep remaining columns unchanged
)


## Defining different models for comparison 

In [294]:
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost Regressor": XGBRegressor(n_estimators=100, random_state=42),
}

In [295]:
best_model = None
best_score = -np.inf

for name, model in models.items():

    # Create pipeline
    pipe = make_pipeline(colmn_trans, StandardScaler(with_mean=False), model)
    
    # Perform 5-Fold Cross-Validation
    scores = cross_val_score(pipe, x_train, y_train, cv=5, scoring='r2')
    
    mean_score = np.mean(scores)
    
    print(f"{name}: \nMean Cross-Validation R² Score = {mean_score:.4f}")
    
    # Select best model
    if mean_score > best_score:
        best_score = mean_score
        best_model = pipe
        best_model_name = name  # Store best model's name

print("\n✅ Best Model:", best_model_name)

#  Train the Best Model on Full Training Data
best_model.fit(x_train, y_train)

#  Test the Best Model
final_score = best_model.score(x_test, y_test)

print("\nFinal Selected Model:", best_model.steps[-1][0])
print(f"Final Test R² Score: {final_score:.4f}")

Linear Regression: 
Mean Cross-Validation R² Score = 0.8035
Decision Tree Regressor: 
Mean Cross-Validation R² Score = 0.6721
Random Forest Regressor: 
Mean Cross-Validation R² Score = 0.7607
XGBoost Regressor: 
Mean Cross-Validation R² Score = 0.7144

✅ Best Model: Linear Regression

Final Selected Model: linearregression
Final Test R² Score: 0.8177


In [296]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import numpy as np

y_pred = best_model.predict(x_test)
y_true = y_test

# 1. Mean Absolute Error (MAE)
mae = mean_absolute_error(y_true, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# 2. Mean Squared Error (MSE)
mse = mean_squared_error(y_true, y_pred)
print(f"Mean Squared Error (MSE): {mse:.4f}")

# 3. Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# 4. Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f}%")

# 5. Explained Variance Score
evs = explained_variance_score(y_true, y_pred)
print(f"Explained Variance Score: {evs:.4f}")

# 6. Adjusted R²
r2 = r2_score(y_true, y_pred)
n = len(y_true)  # number of samples
p = x_test.shape[1]  # number of predictors/features
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f"Adjusted R²: {adjusted_r2:.4f}")


Mean Absolute Error (MAE): 20.7057
Mean Squared Error (MSE): 1341.7307
Root Mean Squared Error (RMSE): 36.6296
Mean Absolute Percentage Error (MAPE): 24.0562%
Explained Variance Score: 0.8182
Adjusted R²: 0.8173


In [297]:
import pickle
pickle.dump(best_model, open('..\model\BestModel.pkl', 'wb'))  